In [24]:
import requests
import pandas as pd
from io import StringIO
response = requests.get('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=20220527&type=ALLBUT0999&_=1653807330929')

response.encoding = 'big5'
lines = response.text.split('\n')
newlines = []
for line in lines:
    # 用「",」切開每一行，看是否被切成17個
    if len(line.split('",')) == 17:
    # 將 line 加到新的 newlines 中
        newlines.append(line)



df = pd.read_csv(StringIO('\n'.join(newlines).replace('=','')))
#讀取檔案，並用虛擬記憶體做出strinIO，使其可以讀成dataFrame

df = df.astype(str)
#先轉乘str才能做下面的replace

df = df.apply(lambda s:s.str.replace(',',''))
#apply也可以改用applymap
df = df.set_index('證券代號')

df = df.apply(lambda s: pd.to_numeric(s, errors='coerce'))
#將整份再度改成數值，不能改的就略過，假如無法轉成數字，則用 NaN 取代

#df.columns[df.isnull().sum() == len(df)])
#從資料欄位推論true,false，並回傳正確欄位值
print(df.isnull().sum() != len(df))
print(df.columns[df.isnull().sum() != len(df)])
df = df[df.columns[df.isnull().sum() != len(df)]]
#df.dropna(axis=1, how='all', inplace=True)
#也可以使用這條
# 要刪除沒有用的columns
# 其中 axis=1 為是說每條columns去檢查有沒有NaN
# how='all' 是說假如全部都是 NaN 則刪除該 column
df


證券名稱           False
成交股數            True
成交筆數            True
成交金額            True
開盤價             True
最高價             True
最低價             True
收盤價             True
漲跌(+/-)        False
漲跌價差            True
最後揭示買價          True
最後揭示買量          True
最後揭示賣價          True
最後揭示賣量          True
本益比             True
Unnamed: 16    False
dtype: bool
Index(['成交股數', '成交筆數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '最後揭示買價',
       '最後揭示買量', '最後揭示賣價', '最後揭示賣量', '本益比'],
      dtype='object')


,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
證券代號,,,,,,,,,,,,,
0050,11895801,10471,1491077052,124.15,125.80,124.05,125.50,2.60,125.50,6,125.55,57,0.00
0051,24300,139,1369689,56.20,56.40,56.20,56.35,0.60,56.20,1,56.35,23,0.00
0052,347836,528,36464371,104.30,105.10,104.20,105.00,2.30,105.00,9,105.05,1,0.00
0053,8568,536,514056,60.10,60.30,59.75,60.30,1.30,60.05,1,60.50,13,0.00
0055,500030,547,11716164,23.20,23.56,23.18,23.56,0.48,23.56,10,23.58,1,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9944,73345,86,1483660,20.20,20.30,20.15,20.30,0.10,20.25,2,20.30,4,4.06
9945,4340664,3429,316042494,72.40,73.20,72.10,72.80,0.80,72.70,61,72.80,39,10.93
9946,17354,25,219913,12.70,12.70,12.60,12.65,0.10,12.65,2,12.70,3,18.07


In [25]:
#選擇當然漲5%以上之股票
df = df[df['收盤價']/df['開盤價']>1.05]
df

,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
證券代號,,,,,,,,,,,,,
1213,117100,56,973154,7.95,8.74,7.82,8.39,0.44,8.0,1,8.39,1,0.00
1560,40591718,24639,5973977334,142.50,152.50,141.50,152.50,13.50,152.5,27678,NaN,0,25.94
2233,5074140,3505,577335143,108.00,116.00,108.00,115.00,8.50,115.0,93,115.50,36,11.65
2483,5036408,1691,128518919,23.65,25.60,23.65,25.60,2.30,25.6,268,NaN,0,14.07
3296,964468,490,15941001,16.05,17.50,15.95,17.40,1.40,17.2,20,17.40,59,0.00
6415,567476,1032,1539286835,2620.00,2775.00,2580.00,2775.00,230.00,2775.0,7,2780.00,2,41.60
6719,2119496,2615,1114410613,507.00,542.00,506.00,542.00,48.50,542.0,3798,NaN,0,38.12
8478,8670012,6620,1627948182,180.00,193.00,179.50,193.00,17.50,193.0,15503,NaN,0,13.78


存成ＣＳＶ檔

In [26]:
df.to_csv('股票練習存成csv.csv',encoding = 'utf_8_sig' )
df = pd.read_csv('股票練習存成csv.csv',index_col = ['證券代號'])
df
df = df.loc[df['收盤價']!=8.39]
df

,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
證券代號,,,,,,,,,,,,,
1560,40591718,24639,5973977334,142.50,152.5,141.50,152.5,13.5,152.5,27678,NaN,0,25.94
2233,5074140,3505,577335143,108.00,116.0,108.00,115.0,8.5,115.0,93,115.5,36,11.65
2483,5036408,1691,128518919,23.65,25.6,23.65,25.6,2.3,25.6,268,NaN,0,14.07
3296,964468,490,15941001,16.05,17.5,15.95,17.4,1.4,17.2,20,17.4,59,0.00
6415,567476,1032,1539286835,2620.00,2775.0,2580.00,2775.0,230.0,2775.0,7,2780.0,2,41.60
6719,2119496,2615,1114410613,507.00,542.0,506.00,542.0,48.5,542.0,3798,NaN,0,38.12
8478,8670012,6620,1627948182,180.00,193.0,179.50,193.0,17.5,193.0,15503,NaN,0,13.78


存到Sqlite中

In [29]:
import sqlite3
連接器 = sqlite3.connect('SQ_當日收盤資料擷取.sqlite3')
df.to_sql('這是自定的table名稱',連接器, if_exists = 'replace')

從Sqlite中取出

In [30]:
df = pd.read_sql('select * from 這是自定的table名稱',連接器,index_col=['證券代號'])
df

,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
證券代號,,,,,,,,,,,,,
1560,40591718,24639,5973977334,142.50,152.5,141.50,152.5,13.5,152.5,27678,NaN,0,25.94
2233,5074140,3505,577335143,108.00,116.0,108.00,115.0,8.5,115.0,93,115.5,36,11.65
2483,5036408,1691,128518919,23.65,25.6,23.65,25.6,2.3,25.6,268,NaN,0,14.07
3296,964468,490,15941001,16.05,17.5,15.95,17.4,1.4,17.2,20,17.4,59,0.00
6415,567476,1032,1539286835,2620.00,2775.0,2580.00,2775.0,230.0,2775.0,7,2780.0,2,41.60
6719,2119496,2615,1114410613,507.00,542.0,506.00,542.0,48.5,542.0,3798,NaN,0,38.12
8478,8670012,6620,1627948182,180.00,193.0,179.50,193.0,17.5,193.0,15503,NaN,0,13.78
